# ManiSkill ACT - Results Viewer

View training metrics, replay demos, and inference videos — all from this notebook.

**Run `bash train_act.sh` first**, then come here to see everything.

In [16]:
from IPython.display import Video, display
import glob

videos = sorted(glob.glob('/workspace/maniskill_runpod/results/inference_videos/*.mp4'))
print(f"Found {len(videos)} videos")

# Play first 3 videos
for v in videos[:]:
    print(f"\n{os.path.basename(v)}")
    display(Video(v, embed=True, width=480))

Found 10 videos

0.mp4



1.mp4



2.mp4



3.mp4



4.mp4



5.mp4



6.mp4



7.mp4



8.mp4



9.mp4


---
## 1. Inference Results & Metrics

The most important output — how well did the trained policy actually perform?

In [17]:
metrics_path = os.path.join(RESULTS_DIR, 'inference_videos', 'metrics.json')

if os.path.exists(metrics_path):
    with open(metrics_path) as f:
        m = json.load(f)
    
    display(HTML(f"""
    <div style='background:#1a1a2e; color:#eee; padding:20px; border-radius:12px; font-family:monospace; max-width:500px;'>
        <h3 style='color:#00d2ff; margin-top:0;'>ACT Policy Results</h3>
        <table style='color:#eee; font-size:15px;'>
            <tr><td style='padding:4px 16px 4px 0; color:#aaa;'>Task</td><td><b>{m.get('env_id','—')}</b></td></tr>
            <tr><td style='padding:4px 16px 4px 0; color:#aaa;'>Demos</td><td>{m.get('num_demos','—')}</td></tr>
            <tr><td style='padding:4px 16px 4px 0; color:#aaa;'>Training iters</td><td>{m.get('total_iters','—'):,}</td></tr>
            <tr><td style='padding:4px 16px 4px 0; color:#aaa;'>Weights</td><td>{m.get('weight_type','—')}</td></tr>
            <tr><td colspan='2'><hr style='border-color:#333;'></td></tr>
            <tr><td style='padding:4px 16px 4px 0; color:#aaa;'>✅ Success Once</td><td style='color:#00ff88; font-size:20px;'><b>{m['success_once']*100:.1f}%</b></td></tr>
            <tr><td style='padding:4px 16px 4px 0; color:#aaa;'>✅ Success at End</td><td style='color:#00ff88;'><b>{m['success_at_end']*100:.1f}%</b></td></tr>
            <tr><td style='padding:4px 16px 4px 0; color:#aaa;'>Avg Return</td><td>{m['avg_return']:.2f}</td></tr>
            <tr><td style='padding:4px 16px 4px 0; color:#aaa;'>Avg Ep Length</td><td>{m['avg_episode_length']:.1f} steps</td></tr>
        </table>
    </div>
    """))
else:
    print(f'⚠ No metrics found at {metrics_path}')
    print('  Run train_act.sh first!')

⚠ No metrics found at /workspace/maniskill_runpod/results/inference_videos/metrics.json
  Run train_act.sh first!


---
## 2. Inference Videos

Watch the trained policy trying to pick up the cube. Green = success, Red = fail.

In [14]:
# Find and display inference videos
inference_videos = sorted(glob.glob(os.path.join(INFERENCE_DIR, "**/*.mp4"), recursive=True))

if inference_videos:
    print(f"Found {len(inference_videos)} inference videos. Showing first 5:\n")
    for v in inference_videos[:5]:
        name = os.path.basename(v)
        size_mb = os.path.getsize(v) / (1024*1024)
        print(f"🎬 {name} ({size_mb:.1f} MB)")
        try:
            display(Video(v, embed=True, width=480))
        except Exception as e:
            print(f"   Could not embed video: {e}")
            print(f"   File path: {v}")
        print()
else:
    print("No inference videos found.")
    print(f"Expected location: {INFERENCE_DIR}")
    
    # Search more broadly
    all_mp4 = glob.glob("/workspace/ManiSkill/examples/baselines/act/runs/**/*.mp4", recursive=True)
    if all_mp4:
        print(f"\nFound {len(all_mp4)} videos in training runs directory:")
        for v in all_mp4[:5]:
            print(f"  {v}")

Found 50 inference videos. Showing first 5:

🎬 0.mp4 (0.1 MB)



🎬 1.mp4 (0.1 MB)



🎬 10.mp4 (0.1 MB)



🎬 11.mp4 (0.1 MB)



🎬 12.mp4 (0.1 MB)


---
## 3. Replay Demo Videos

These are the expert demonstrations the policy was trained on (rendered to RGBD).

In [4]:
replay_videos = sorted(glob.glob(os.path.join(REPLAY_DIR, "*.mp4")))

if replay_videos:
    print(f"Found {len(replay_videos)} replay demo videos. Showing first 3:\n")
    for v in replay_videos[:3]:
        name = os.path.basename(v)
        print(f"🎬 {name}")
        try:
            display(Video(v, embed=True, width=480))
        except Exception as e:
            print(f"   Could not embed: {e}")
        print()
else:
    print("No replay videos found.")
    
    # Search in default maniskill location
    demo_vids = glob.glob(os.path.expanduser("~/.maniskill/demos/PickCube-v1/**/*.mp4"), recursive=True)
    if demo_vids:
        print(f"Found {len(demo_vids)} videos in ~/.maniskill/demos/:")
        for v in demo_vids[:3]:
            print(f"  {v}")
            display(Video(v, embed=True, width=480))
            print()

Found 101 replay demo videos. Showing first 3:

🎬 0.mp4



🎬 1.mp4



🎬 10.mp4


---
## 4. Training Logs

Check what files were saved during training (checkpoints, tensorboard logs, etc.)

In [5]:
# List training artifacts
if os.path.exists(LOGS_DIR) and os.listdir(LOGS_DIR):
    print("Training artifacts:\n")
    for root, dirs, files in os.walk(LOGS_DIR):
        level = root.replace(LOGS_DIR, '').count(os.sep)
        indent = '  ' * level
        folder = os.path.basename(root)
        if level == 0:
            folder = "training_logs/"
        print(f"{indent}📁 {folder}")
        sub_indent = '  ' * (level + 1)
        for f in sorted(files):
            size = os.path.getsize(os.path.join(root, f))
            if size > 1024*1024:
                size_str = f"{size/(1024*1024):.1f} MB"
            elif size > 1024:
                size_str = f"{size/1024:.0f} KB"
            else:
                size_str = f"{size} B"
            print(f"{sub_indent}📄 {f} ({size_str})")
else:
    print("No training logs found yet.")
    
    # Check ACT runs directory directly
    act_runs = "/workspace/ManiSkill/examples/baselines/act/runs"
    if os.path.exists(act_runs):
        print(f"\nBut found runs at: {act_runs}")
        for item in os.listdir(act_runs):
            print(f"  📁 {item}")

Training artifacts:

📁 training_logs/
  📄 events.out.tfevents.1770203740.9a955fb959af.13649.0 (85 KB)
  📁 videos
    📄 0.mp4 (124 KB)
    📄 1.mp4 (127 KB)
    📄 10.mp4 (126 KB)
    📄 100.mp4 (148 KB)
    📄 101.mp4 (165 KB)
    📄 102.mp4 (146 KB)
    📄 103.mp4 (158 KB)
    📄 104.mp4 (150 KB)
    📄 105.mp4 (180 KB)
    📄 106.mp4 (151 KB)
    📄 107.mp4 (141 KB)
    📄 108.mp4 (163 KB)
    📄 109.mp4 (151 KB)
    📄 11.mp4 (123 KB)
    📄 110.mp4 (158 KB)
    📄 111.mp4 (141 KB)
    📄 112.mp4 (154 KB)
    📄 113.mp4 (160 KB)
    📄 114.mp4 (162 KB)
    📄 115.mp4 (162 KB)
    📄 116.mp4 (160 KB)
    📄 117.mp4 (144 KB)
    📄 118.mp4 (169 KB)
    📄 119.mp4 (165 KB)
    📄 12.mp4 (127 KB)
    📄 120.mp4 (158 KB)
    📄 121.mp4 (160 KB)
    📄 122.mp4 (144 KB)
    📄 123.mp4 (153 KB)
    📄 124.mp4 (152 KB)
    📄 125.mp4 (145 KB)
    📄 126.mp4 (141 KB)
    📄 127.mp4 (162 KB)
    📄 128.mp4 (166 KB)
    📄 129.mp4 (144 KB)
    📄 13.mp4 (132 KB)
    📄 130.mp4 (157 KB)
    📄 131.mp4 (146 KB)
    📄 132.mp4 (164 KB

In [6]:
# List checkpoints
checkpoints = glob.glob(os.path.join(LOGS_DIR, "**/*.pt"), recursive=True)
if not checkpoints:
    checkpoints = glob.glob("/workspace/ManiSkill/examples/baselines/act/runs/**/*.pt", recursive=True)

if checkpoints:
    print(f"Found {len(checkpoints)} checkpoint(s):\n")
    for cp in sorted(checkpoints):
        size_mb = os.path.getsize(cp) / (1024*1024)
        print(f"  💾 {os.path.basename(cp)} ({size_mb:.1f} MB)")
        print(f"     Path: {cp}")
else:
    print("No checkpoints found.")

Found 2 checkpoint(s):

  💾 best_eval_success_at_end.pt (127.3 MB)
     Path: /workspace/maniskill_runpod/results/training_logs/checkpoints/best_eval_success_at_end.pt
  💾 best_eval_success_once.pt (127.3 MB)
     Path: /workspace/maniskill_runpod/results/training_logs/checkpoints/best_eval_success_once.pt


---
## 5. Tensorboard (optional)

For detailed training curves, start tensorboard:

In [7]:
# Uncomment to start tensorboard inline (or run in terminal)
# %load_ext tensorboard
# %tensorboard --logdir /workspace/ManiSkill/examples/baselines/act/runs/ --port 6006

print("To view tensorboard, run in a terminal:")
print("  tensorboard --logdir /workspace/ManiSkill/examples/baselines/act/runs/ --port 6006")
print("\nThen open port 6006 in RunPod's proxy or exposed ports.")

To view tensorboard, run in a terminal:
  tensorboard --logdir /workspace/ManiSkill/examples/baselines/act/runs/ --port 6006

Then open port 6006 in RunPod's proxy or exposed ports.


---
## 6. Re-render with Ray Tracing (optional)

Make prettier videos by replaying inference trajectories with ray-tracing enabled.

In [8]:
# Find inference trajectory file
traj_files = glob.glob(os.path.join(INFERENCE_DIR, "*.h5"))
if not traj_files:
    traj_files = glob.glob("/workspace/ManiSkill/examples/baselines/act/runs/**/*.h5", recursive=True)

if traj_files:
    traj_file = traj_files[0]
    print(f"Found trajectory: {traj_file}")
    print(f"\nTo re-render with ray-tracing, run:")
    print(f"")
    print(f"  python -m mani_skill.trajectory.replay_trajectory \\")
    print(f"      --traj-path={traj_file} \\")
    print(f"      --use-env-states --shader=\"rt-fast\" \\")
    print(f"      --save-video --allow-failure -o \"none\"")
    print(f"")
    print("This will produce photo-realistic videos (slower but much prettier).")
else:
    print("No trajectory files found for re-rendering.")

Found trajectory: /workspace/maniskill_runpod/results/inference/inference_trajectory.h5

To re-render with ray-tracing, run:

  python -m mani_skill.trajectory.replay_trajectory \
      --traj-path=/workspace/maniskill_runpod/results/inference/inference_trajectory.h5 \
      --use-env-states --shader="rt-fast" \
      --save-video --allow-failure -o "none"

This will produce photo-realistic videos (slower but much prettier).


In [11]:
# Uncomment to actually run ray-traced re-rendering
# (takes a while — only do this if you want prettier videos)

if traj_files:
    !python -m mani_skill.trajectory.replay_trajectory \
        --traj-path={traj_files[0]} \
        --use-env-states --shader="rt-fast" \
        --save-video --allow-failure -o "none" \
        --count 5

0step [00:00, ?step/s]2026-02-04 13:24:18,956 - mani_skill  - WARNING - shader_dir argument will be deprecated after ManiSkill v3.0.0 official release. Please use sensor_configs/human_render_camera_configs to set shaders.
  0%|                                                   | 0/5 [00:05<?, ?step/s]
Replayed 5 episodes, 5/5=100.00% demos saved


---
## 7. Quick Summary

In [9]:
# Print a final summary
print("═" * 50)
print("           PIPELINE SUMMARY")
print("═" * 50)

# Metrics
if os.path.exists(metrics_file):
    with open(metrics_file) as f:
        m = json.load(f)
    print(f"\n  Task:          {m['task']}")
    print(f"  Demos used:    {m['num_demos']}")
    print(f"  Training:      {m['total_iters']} iterations")
    print(f"  Success rate:  {m['success_rate']:.1f}%")
    print(f"  Avg reward:    {m['avg_reward']:.2f}")
else:
    print("\n  No results yet — run train_act.sh first")

# File counts
replay_count = len(glob.glob(os.path.join(REPLAY_DIR, "*.mp4")))
infer_count = len(glob.glob(os.path.join(INFERENCE_DIR, "**/*.mp4"), recursive=True))
ckpt_count = len(glob.glob(os.path.join(LOGS_DIR, "**/*.pt"), recursive=True))

print(f"\n  Replay videos:    {replay_count}")
print(f"  Inference videos: {infer_count}")
print(f"  Checkpoints:      {ckpt_count}")
print()
print("═" * 50)

══════════════════════════════════════════════════
           PIPELINE SUMMARY
══════════════════════════════════════════════════

  Task:          PickCube-v1
  Demos used:    100
  Training:      30000 iterations
  Success rate:  0.0%
  Avg reward:    0.18

  Replay videos:    101
  Inference videos: 50
  Checkpoints:      2

══════════════════════════════════════════════════
